In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [6]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
    read_options={"timeout" : 600},

)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 19:46:05,764 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:46:05,777 INFO: Initializing external client
2025-03-03 19:46:05,777 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:46:06,656 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Fetching data from 2025-02-03 00:46:05.764318+00:00 to 2025-03-03 23:46:05.764318+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.92s) 


In [8]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features_and_target
features = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [9]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 19:46:54,970 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:46:54,986 INFO: Initializing external client
2025-03-03 19:46:54,988 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:46:55,752 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680


In [11]:
features, target = features

In [12]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [13]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,196,0.0,2025-03-04 01:00:00+00:00
1,263,74.0,2025-03-04 01:00:00+00:00
2,255,1.0,2025-03-04 01:00:00+00:00
3,210,0.0,2025-03-04 01:00:00+00:00
4,53,0.0,2025-03-04 01:00:00+00:00
...,...,...,...
246,122,0.0,2025-03-04 01:00:00+00:00
247,142,119.0,2025-03-04 01:00:00+00:00
248,231,74.0,2025-03-04 01:00:00+00:00
249,205,0.0,2025-03-04 01:00:00+00:00


In [14]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-03 19:53:30,174 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:53:30,182 INFO: Initializing external client
2025-03-03 19:53:30,182 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:53:31,278 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680


In [15]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214680/fs/1203298/fg/1401912


Uploading Dataframe: 100.00% |██████████| Rows 251/251 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214680/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)